In [40]:
import pandas as pd
import numpy as np

In [44]:
a=pd.read_csv('대분류세분류테이블.csv',encoding='cp949')

In [41]:
b=pd.read_csv('17기존고객.csv',encoding='cp949')

In [42]:
c=pd.read_csv('16기존고객.csv',encoding='cp949')

In [47]:
h=a[(a['소비재분류']=='선매품')]

In [46]:
# 선매품 구매횟수
df_pivot_f1 = pd.pivot_table(h,                # 피벗할 데이터프레임
                     index = '고객번호',    # 행 위치에 들어갈 열
                     columns = 'QUARTER',    # 열 위치에 들어갈 열
                     values = '영수증번호',     # 데이터로 사용할 열
                     aggfunc = 'count')   # 데이터 집계함수

In [48]:
df_pivot_f1 = df_pivot_f1.fillna(0)

In [49]:
# 8기 드랍 테이블
df_pivot_f2 = df_pivot_f1.drop(['8Q'],axis=1)

In [51]:
df_pivot_f2['선매품구매횟수'] = df_pivot_f2['1Q'] + df_pivot_f2['2Q'] + df_pivot_f2['3Q'] + df_pivot_f2['4Q'] + df_pivot_f2['5Q']+ df_pivot_f2['6Q'] + df_pivot_f2['7Q']

In [ ]:
df_pivot_f3 = df_pivot_f2.drop(['1Q','2Q','3Q','4Q','5Q','6Q','7Q'],axis=1)

In [ ]:
# 8기 드랍, 1,2,3,4,5,6,7기 구매횟수 합친 값만 나옴
df_pivot_f3

In [ ]:
# 7기까지 기존고객과 7기까지 총구매횟수 합침
tree=pd.merge(b, df_pivot_f3,on='고객번호', how='left')

In [53]:
# 분기별 구매금액
df_pivot_s4 = pd.pivot_table(h,                # 피벗할 데이터프레임
                     index = '고객번호',    # 행 위치에 들어갈 열
                     columns = 'QUARTER',    # 열 위치에 들어갈 열
                     values = '구매금액',     # 데이터로 사용할 열
                     aggfunc = 'sum')   # 데이터 집계함수

In [54]:
df_pivot_s4 = df_pivot_s4.fillna(0)

In [55]:
df_pivot_s5 = df_pivot_s4.drop(['8Q'],axis=1)

In [56]:
# 선매품 등급화
aa=df_pivot_s5['1Q'].sort_values(ascending=False)
cuts = pd.qcut(aa,10,labels=np.arange(1,11)[::-1])
cc=pd.concat([aa,cuts],axis=1)

In [57]:
lst=['2Q','3Q','4Q','5Q','6Q','7Q']
for i in lst:
    aa=df_pivot_s5[f'{i}'].sort_values(ascending=False)
    
    cuts = pd.qcut(aa,10,labels=np.arange(1,11)[::-1])
    table=pd.concat([aa,cuts],axis=1)  
    cc=pd.merge(cc,table, on='고객번호',how='left')

In [ ]:
cc.columns=['선매품1','선매품등급1','선매품2','선매품등급2','선매품3','선매품등급3','선매품4','선매품등급4','선매품5','선매품등급5','선매품6','선매품등급6''선매품7','선매품등급7']

In [22]:
cc = cc.drop(['선매품1','선매품2','선매품3','선매품4','선매품5','선매품6','선매품7'],axis=1)

In [23]:
cc = cc.reset_index()

In [24]:
cc = cc.sort_values('고객번호')

In [ ]:
# 7기까지 기존고객과 합침
tree2=pd.merge(b, cc,on='고객번호', how='left')
tree2

In [33]:
# 증감을 구하기 위해 엑셀로 옮김
import csv
tree2.to_excel('선매품17.xlsx',encoding='euc-kr',index=False)

In [33]:
# 구한 증감을 다시 불러옴
z= pd.read_excel('선매품17.xlsx')

In [36]:
z = z.drop(['선매품등급1','선매품등급2','선매품등급3','선매품등급4','선매품등급5','선매품등급6','선매품등급7'],axis=1)

In [ ]:
tree3=pd.merge(tree, z,on='고객번호', how='left')

In [37]:
import csv
tree3.to_csv('17선매품.csv',mode="w",encoding='euc-kr',index=False)